# Fn to FC
> python module to convert a given Fn into FC automatically

In [ ]:
#| default_exp core.fn_to_fc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# Importing openai and our custom functions
import openai
import json
import ast
import inspect
import importlib
import warnings

from typing import Optional, Union, Callable, Literal,  Tuple
from types import NoneType
from llmcam.ytlive import YTLive, NHsta
from llmcam.gpt4v import ask_gpt4v

import textwrap
from colorama import Fore, Back, Style

## Example functions

For our first MVP, response generation mostly concern with GPT models answering generic questions and using a single tool for capturing and extracting information from a Youtube Livestream.

In [ ]:
import glob
from llmcam.ytlive import YTLive, NHsta
from llmcam.gpt4v import ask_gpt4v

In [ ]:
#| export
def capture_youtube_live_frame_and_save(
        link: Optional[str] = None,  # YouTube Live link
        place: Optional[str] = None,  # Location of live image
    ) -> str:  # Path to the saved image
    """Capture a jpeg file from YouTube Live and save in data directory"""
    if link is not None:
        live = YTLive(url=link, place=place)
    
    else:
        live = NHsta()
    return str(live())

In [ ]:
show_doc(capture_youtube_live_frame_and_save)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/fn_to_fc.py#L28){target="_blank" style="float:right; font-size:smaller"}

### capture_youtube_live_frame_and_save

>      capture_youtube_live_frame_and_save (link:Optional[str]=None,
>                                           place:Optional[str]=None)

*Capture a jpeg file from YouTube Live and save in data directory*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| link | Optional | None | YouTube Live link |
| place | Optional | None | Location of live image |
| **Returns** | **str** |  | **Path to the saved image** |

In [ ]:
#| export
def ask_gpt4v_about_image_file(
        path:str  # Path to the image file
    ) -> str:  # JSON string with quantitative information
    """Tell all about quantitative information from a given image file"""
    info = ask_gpt4v(path)
    return json.dumps(info)

In [ ]:
show_doc(ask_gpt4v_about_image_file)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/fn_to_fc.py#L41){target="_blank" style="float:right; font-size:smaller"}

### ask_gpt4v_about_image_file

>      ask_gpt4v_about_image_file (path:str)

*Tell all about quantitative information from a given image file*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| path | str | Path to the image file |
| **Returns** | **str** | **JSON string with quantitative information** |

## Utilities for GPT Function calling

We can use dynamic utilities functions to integrate this to GPT Function calling:  

- Parmater descriptions: extract parameter descriptions from a funcion
- Parameter converter: convert Python parameter types into schema accepted formats
- Schema generator: extract function information into tool schema to bet set for GPT
- Function execution: execute function dynamically based on function names and input arguments

### 1. Extractor for parameter descriptions

In [ ]:
#| export
# Extract parameter comments from the function
def extract_parameter_comments(
        func: Callable  # Function to extract comments from
    ) -> dict[str, str]:  # Dictionary with parameter comments
    """Extract comments for function arguments"""
    # Get the source code of the function
    source = inspect.getsource(func)
    # Parse the source code into an AST
    tree = ast.parse(source)
    
    # Extract comments for function arguments
    comments = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef) and node.name == func.__name__:
            # Get arguments and comments in the function
            for arg in node.args.args:
                arg_name = arg.arg
                # Check if there's an inline comment associated with the argument
                if arg.end_lineno and arg.col_offset:
                    # Loop through the source code lines to find the comment
                    lines = source.splitlines()
                    for line in lines:
                        if line.strip().startswith(f"{arg_name}:") and "#" in line:
                            comment = line.split("#")[1].strip()
                            comments[arg_name] = comment
    return comments

In [ ]:
show_doc(extract_parameter_comments)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/fn_to_fc.py#L50){target="_blank" style="float:right; font-size:smaller"}

### extract_parameter_comments

>      extract_parameter_comments (func:Callable)

*Extract comments for function arguments*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| func | Callable | Function to extract comments from |
| **Returns** | **dict** | **Dictionary with parameter comments** |

Test usage with example functions:

In [ ]:
print(f"Parameters of capture Youtube Live frame function: \
{extract_parameter_comments(capture_youtube_live_frame_and_save)}")
print(f"Parameters of ask GPT4V about image file function: \
{extract_parameter_comments(ask_gpt4v_about_image_file)}")

Parameters of capture Youtube Live frame function: {'link': 'YouTube Live link', 'place': 'Location of live image'}
Parameters of ask GPT4V about image file function: {'path': 'Path to the image file'}


In [ ]:
#| hide
# Test the functions
assert extract_parameter_comments(capture_youtube_live_frame_and_save) == {'link': 'YouTube Live link', 'place': 'Location of live image'}
assert extract_parameter_comments(ask_gpt4v_about_image_file) == {'path': 'Path to the image file'}

### 2. Converter for Python parameter types to acceptable tool schema

In [ ]:
#| export
def param_converter(
        param_type,  # The type of the parameter
        description  # The description of the parameter
    ) -> dict:  # The converted parameter
    """Convert Python parameter types to acceptable types for tool schema"""
    simple_types = {
        str: "string",
        int: "number",
        float: "number",
        bool: "boolean",
    }
    if param_type in simple_types:
        return { "type": simple_types[param_type], "description": description }
    elif param_type == NoneType:
        return { "type": "null", "description": "A default value will be automatically used." }
    
    if param_type == list or getattr(param_type, "__origin__", None) == list:
        item_type = param_type.__args__[0] if hasattr(param_type, "__args__") and param_type.__args__ else str
        return {
            "type": "array",
            "description": description,
            "items": { "type": param_converter(item_type, description)["type"] }
        }
    
    if hasattr(param_type, '__origin__') and param_type.__origin__ == Union:
        # Recursively convert the types
        descriptions = description.split(" or ")
        subtypes = param_type.__args__
        if len(subtypes) > len(descriptions):
            descriptions = descriptions + ["A description is not provided"] * (len(subtypes) - len(descriptions))

        return {
            "anyOf": [param_converter(subtype, desc) for subtype, desc in zip(subtypes, descriptions)]
        }
    return { "type": "string", "description": description }

Test usage with a more complicated data type `Optional[str]`:

In [ ]:
param_schema = param_converter(Optional[str], "YouTube Live link")
print(json.dumps(param_schema, indent=2))

{
  "anyOf": [
    {
      "type": "string",
      "description": "YouTube Live link"
    },
    {
      "type": "null",
      "description": "A default value will be automatically used."
    }
  ]
}


In [ ]:
#| hide
# Test the function
assert param_schema == { "anyOf": [
        {
            "type": "string",
            "description": "YouTube Live link"
        },
        {
            "type": "null",
            "description": "A default value will be automatically used."
        }
    ]
}

### 3. Tool schema

In [ ]:
#| export
def tool_schema(
        func: Callable,  # The function to generate the schema for
        service_name: Optional[str] = None,  # The name of the service
    ) -> dict:  # The generated tool schema
    """Automatically generate a schema from its parameters and docstring"""
    # Extract function name, docstring, and parameters
    func_name = func.__name__
    func_description = func.__doc__ or "No description provided."
    func_module = func.__module__
    signature = inspect.signature(func)
    
    # Create parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    # Populate properties and required fields
    param_comments = extract_parameter_comments(func)
    for param_name, param in signature.parameters.items():
        param_type = param.annotation if param.annotation != inspect._empty else str
        
        # Add parameter to schema
        parameters_schema["properties"][param_name] = param_converter(
            param_type, 
            param_comments.get(param_name, "No description provided.")
        )
        
        # Mark as required if no default
        if param.default == inspect.Parameter.empty:
            parameters_schema["required"].append(param_name)
    
    # Build final tool schema
    tool_schema = {
        "type": "function",
        "function": {
            "name": func_name,
            "description": func_description,
            "parameters": parameters_schema,
            "metadata": {
                "module": func_module,
                "service": service_name or func_module
            }
        }
    }
    
    return tool_schema

In [ ]:
show_doc(tool_schema)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/fn_to_fc.py#L114){target="_blank" style="float:right; font-size:smaller"}

### tool_schema

>      tool_schema (func:Callable, service_name:Optional[str]=None)

*Automatically generate a schema from its parameters and docstring*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| func | Callable |  | The function to generate the schema for |
| service_name | Optional | None | The name of the service |
| **Returns** | **dict** |  | **The generated tool schema** |

In [ ]:
#| export
# Environmental setting up
YTLiveTools = [tool_schema(fn) for fn in (capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file)]

In [ ]:
#| echo: false
print(json.dumps(YTLiveTools, indent=2))

[
  {
    "type": "function",
    "function": {
      "name": "capture_youtube_live_frame_and_save",
      "description": "Capture a jpeg file from YouTube Live and save in data directory",
      "parameters": {
        "type": "object",
        "properties": {
          "link": {
            "anyOf": [
              {
                "type": "string",
                "description": "YouTube Live link"
              },
              {
                "type": "null",
                "description": "A default value will be automatically used."
              }
            ]
          },
          "place": {
            "anyOf": [
              {
                "type": "string",
                "description": "Location of live image"
              },
              {
                "type": "null",
                "description": "A default value will be automatically used."
              }
            ]
          }
        },
        "required": []
      },
      "metadata": {
        "mod

In [ ]:
#| hide
# Test the function
# Check the schema of the first tool
assert YTLiveTools[0]["function"]["name"] == "capture_youtube_live_frame_and_save"
assert YTLiveTools[0]["function"]["description"] == "Capture a jpeg file from YouTube Live and save in data directory"
assert YTLiveTools[0]["function"]["parameters"]["properties"]["link"] == {
    "anyOf": [
        {
            "type": "string",
            "description": "YouTube Live link"
        },
        {
            "type": "null",
            "description": "A default value will be automatically used."
        }
    ]
}

# Check the schema of the second tool
assert YTLiveTools[1]["function"]["name"] == "ask_gpt4v_about_image_file"
assert YTLiveTools[1]["function"]["description"] == "Tell all about quantitative information from a given image file"
assert YTLiveTools[1]["function"]["parameters"]["properties"]["path"] == {
    "type": "string",
    "description": "Path to the image file"
}
assert YTLiveTools[1]["function"]["parameters"]["required"] == ["path"]

### 4. Excecution functions

In [ ]:
#| export
# Support functions to handle tool response,where call == response.choices[0].message.tool_calls[i]
def fn_name(call): return call["function"]["name"]
def fn_args(call): return json.loads(call["function"]["arguments"])
def fn_metadata(tool): return tool["function"]["metadata"]

def fn_exec(call, tools=[]):
    """Execute the function call"""
    for tool in tools:
        # Check if the function name matches
        if call['function']['name'] != tool['function']['name']:
            continue

        # Execute the function by dynamically importing the module
        try:
            module_path = tool['function']['metadata']['module']
            module = importlib.import_module(module_path)
            fn = getattr(module, fn_name(call))
            return fn(**fn_args(call))
        
        # If the function is not found, try to fix it
        except Exception as e:
            if not 'fixup' in tool['function']:
                continue
            module_path, fn_path = tool['function']['fixup'].rsplit('.', 1)
            fn = getattr(importlib.import_module(module_path), fn_path)
            return fn(fn_name(call), **fn_metadata(tool), **fn_args(call))

def fn_result_content(call, tools=[]):
    """Create a content containing the result of the function call"""
    content = dict()
    content.update(fn_args(call))
    content.update({fn_name(call): fn_exec(call, tools)})
    return json.dumps(content)

In [ ]:
#| export
def print_msg(msg):
    who = msg['role'].capitalize()
    who = (Fore.RED if who in "System" else Fore.GREEN if who in "User" else Fore.BLUE if who in "Assistant" else Fore.CYAN) + who
    who = Back.YELLOW + who
    print(Style.BRIGHT + Fore.RED + f">> {who}:" + Style.RESET_ALL)
    try:
        print(textwrap.fill(msg["content"], 100))
    except:
        print(msg)

def print_msgs(msgs, with_tool=False):
    for msg in msgs:
        if not with_tool and any(key in msg for key in ('tool_calls', 'tool_call_id')):
            continue
        print_msg(msg)

def form_msg(
    role: Literal["system", "user", "assistant", "tool"],  # The role of the message sender
    content: str,  # The content of the message
    tool_call_id: Optional[str] = None,  # The ID of the tool call (if role == "tool")
):
    """Create a message for the conversation"""
    msg = {
        "role": role,
        "content": content
    }
    if role == "tool":
        msg["tool_call_id"] = tool_call_id
    return msg

def form_msgs(msgs): return [{"role":m[0],"content":m[1]} for m in msgs]        

In [ ]:

#| export
def complete(
        messages: list[dict],  # The list of messages
        tools: list[dict] = [],  # The list of tools
    ) -> Tuple[str, str]:  # The role and content of the last message
    """Complete the conversation with the given message"""
    # Generate the response from GPT-4
    response = openai.chat.completions.create(model="gpt-4o", messages=messages, tools=tools)
    res = response.choices[0].message
    messages.append(res.to_dict())

    # Handle the tool response
    for call in res.to_dict().get('tool_calls', []):
        # Append the tool response to the list
        messages.append(
            form_msg(
                role="tool",
                content=fn_result_content(call, tools=tools),
                tool_call_id=call["id"]
            )
        )

    if res.to_dict().get('tool_calls'):
        # Recursively call the complete function to handle the tool response
        complete(
            messages, 
            tools=tools
        )

    # Return the last message
    return messages[-1]['role'], messages[-1]['content']

In [ ]:
show_doc(complete)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/fn_to_fc.py#L236){target="_blank" style="float:right; font-size:smaller"}

### complete

>      complete (messages:list[dict], tools:list[dict]=[])

*Complete the conversation with the given message*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| messages | list |  | The list of messages |
| tools | list | [] | The list of tools |
| **Returns** | **Tuple** |  | **The role and content of the last message** |

Test with our existing toolbox:

In [ ]:
#| eval: false
# Test the function
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to assist the user."),
    ("user", "Hi, can you capture YouTube Live?")
])
complete(messages, YTLiveTools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Hi, can you capture YouTube Live?
>> Assistant:
Yes, I can capture a frame from a YouTube Live stream. Please provide the link to the YouTube Live
stream you want captured.


Let's try to continue this conversation:

In [ ]:
#| eval: false
messages.append(form_msg("user", "You can use the default link."))
complete(messages, YTLiveTools)
print_msgs(messages)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
09.12.2024 21:04:02 Olympiaterminaali
>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Hi, can you capture YouTube Live?
>> Assistant:
Yes, I can capture a frame from a YouTube Live stream. Please provide the link to the YouTube Live
stream you want captured.
>> User:
You can use the default link.
>> Assistant:
I have successfully captured a frame from the default YouTube Live stream and saved it as
"Olympiaterminaali.jpg". If you need any further information or analysis on this image, feel free to
ask!


In [ ]:
#| eval: false
messages.append(form_msg("user", "Can you extract information from this file?"))
complete(messages, YTLiveTools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Hi, can you capture YouTube Live?
>> Assistant:
Yes, I can capture a frame from a YouTube Live stream. Please provide the link to the YouTube Live
stream you want captured.
>> User:
You can use the default link.
>> Assistant:
I have successfully captured a frame from the default YouTube Live stream and saved it as
"Olympiaterminaali.jpg". If you need any further information or analysis on this image, feel free to
ask!
>> User:
Can you extract information from this file?
>> Assistant:
Here's the information extracted from the image:  - **Timestamp**: December 9, 2024, at 21:04:02 -
**Location**: Olympiaterminaali - **Image Dimensions**: 1280 x 720 pixels - **Buildings**:
Approximately 5 buildings, with height ranging from 3 to 5 stories - **Vehicles**:   - Cars: 15   -
Trucks: 2   - Boats: 2 - **Parking**: 20 available parking spaces - **Street Lights**: 25 -
**People**: 3 - **Time of 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()